In [62]:
!pip install vectorbt numba yfinance pandas_ta

     |████████████████████████████████| 115 kB 5.0 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=23c0a5468c22321acb06b3dfc13851604fe5af5472f0994686c6fc51b283adc8
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
Successfully built pandas-ta


In [44]:
!git clone https://github.com/tthwmk/Code.git

Cloning into 'Code'...
remote: Enumerating objects: 2238, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 2238 (delta 22), reused 143 (delta 19), pack-reused 2088
Receiving objects: 100% (2238/2238), 408.51 MiB | 30.42 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [45]:
!ls

Code  sample_data


In [4]:
import vectorbt as vbt
from vectorbt.signals.factory import SignalFactory
import numpy as np
from numba import njit
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from abc import abstractmethod, ABCMeta
import time
from pathlib import Path
from itertools import product, combinations, permutations 
from typing import List, Optional, Union
import time
import yfinance as yf

## Top 30 Stocks in India

In [5]:
tickers = [
    "NTPC.NS","HINDALCO.NS","BHARTIARTL.NS",
    "SHREECEM.NS","TCS.NS","HDFCLIFE.NS",
    "CIPLA.NS","LT.NS","ULTRACEMCO.NS",
    "WIPRO.NS","KOTAKBANK.NS","BAJFINANCE.NS",
    "TATACONSUM.NS","BAJAJFINSV.NS","MARUTI.NS",
    "TITAN.NS","ICICIBANK.NS","ONGC.NS","ITC.NS",
    "APOLLOHOSP.NS","BRITANNIA.NS","BAJAJ-AUTO.NS",
    "TECHM.NS","COALINDIA.NS","TATASTEEL.NS","HEROMOTOCO.NS",
    "INDUSINDBK.NS","NESTLEIND.NS","M&M.NS","RELIANCE.NS"
]

### Utilities

In [46]:
DATA_PATH = Path("./Code/data").resolve()

def download_data(
    ticker: str,
    period: Optional[str] = "max",
    interval: Optional[str] = "1d",
    auto_adjust: Optional[bool] = True,
    path: Union[str, Path] = DATA_PATH,
):
    """
    downloads data from yahoo and converts it to parquet.
    to be run using a threadpool executor since it is IO bound.

    i.e from concurrent.futures import ThreadPoolExecutor
    with ThreadPoolExecutor as executor:
        executor.map(download_data, [Tickers])

    TODO:
        stop it from saving parquet files of empty tickers

    """
    try:
        temp = yf.Ticker(ticker)
        temp_history = temp.history(
            period=period, interval=interval, auto_adjust=auto_adjust
        )
        temp_history.to_parquet(
            path=f"{path/ticker}.parquet", engine="pyarrow", index=True
        )
    except:
        pass
    else:
        print(f"{ticker} was successfully downloaded")
        time.sleep(2)


def delete_empty_parquet(path):
    for p in path.iterdir():
        if (len(pd.read_parquet(p))) == 0:
            print(f"deleting {p}")
            p.unlink()


def get_file_name(path: Path, output: list = []):
    for file in path.iterdir():
        output.append(file.stem)
    return output

def permute_data(data: pd.Series, seed: Optional[int] = 1730) -> pd.Series:
    """
    shuffled a given data, N.B faster than the np.shuffle and
    allows for using random state for replicability
    """
    rng = np.random.default_rng(seed)
    permuted_data = rng.permutation(data.values)  # faster than choice
    return pd.Series(data=permuted_data, index=data.index)


def random_like(
    data: pd.Series,
    seed: Optional[int] = 1730,
    p: List[float] = [
        0.5,
    ],
    binomial: Optional[bool] = True,
) -> pd.Series:
    """
    generates entries of the same size as the main entries according to given probabilities.
    two options are given, Binomial or choice. Binomial is faster

    """
    rng = np.random.default_rng(seed)
    if binomial:
        random_data = rng.binomial(n=1, p=0.5, size=data.size).astype("bool")
    else:
        random_data = rng.choice([True, False], size=data.size, p=p)
    return pd.Series(data=random_data, index=data.index)

## Download Data

In [47]:
with ThreadPoolExecutor() as executor:
    executor.map(download_data, tickers)

HDFCLIFE.NS was successfully downloaded
NTPC.NS was successfully downloaded
TCS.NS was successfully downloaded
ULTRACEMCO.NS was successfully downloaded
BHARTIARTL.NS was successfully downloaded
LT.NS was successfully downloaded
SHREECEM.NS was successfully downloaded
WIPRO.NS was successfully downloaded
HINDALCO.NS was successfully downloaded
CIPLA.NS was successfully downloaded
KOTAKBANK.NS was successfully downloaded
BAJFINANCE.NS was successfully downloaded
BAJAJFINSV.NS was successfully downloaded
MARUTI.NS was successfully downloaded
TITAN.NS was successfully downloaded
ITC.NS was successfully downloaded
TATACONSUM.NS was successfully downloaded
ICICIBANK.NS was successfully downloaded
APOLLOHOSP.NS was successfully downloaded
ONGC.NS was successfully downloaded
BRITANNIA.NS was successfully downloaded
BAJAJ-AUTO.NS was successfully downloaded
COALINDIA.NS was successfully downloaded
TECHM.NS was successfully downloaded
NESTLEIND.NS was successfully downloaded
HEROMOTOCO.NS was s

In [48]:
START_DATE = pd.to_datetime("2009-10-01")
END_DATE = pd.to_datetime("2020-01-31")

## SETTINGS

### Base strategy class

In [49]:
class Strategy(metaclass=ABCMeta):
    @abstractmethod
    def get_entries(self):
        """
        returns entries signals
        """
        pass

    @abstractmethod
    def get_exits(self):
        """
        returns exit signals
        """
        pass

### Backtest

In [50]:
class Backtest:
    def __init__(
        self,
        ticker: str,
        strategy: Strategy,
        fees: float = 0.001,
        size: float = np.inf,
        freq: str = "1D",
    ):
        self.ticker = ticker
        self.strategy = strategy
        self.fees = fees
        self.size = size
        self.freq = freq

    def __repr__(self):
        return f"<Bactest {str(self.strategy)}>"

    def _get_portfolio(self, entries, exits):
        return vbt.Portfolio.from_signals(
            close=self.strategy.data,
            entries=entries,
            exits=exits,
            fees=self.fees,
            size=self.size,
            freq=self.freq,
        )

    def run(self, **kwargs):
        entries = self.strategy.get_entries()
        exits = self.strategy.get_exits()
        
        
        

        random_entries = entries.apply(random_like)
        permuted_entries = entries.apply(permute_data)

        portfolio = self._get_portfolio(entries, exits, **kwargs)
        random_porfolio = self._get_portfolio(random_entries, exits, **kwargs)
        permuted_portfolio = self._get_portfolio(permuted_entries, exits, **kwargs)

        output = {
            self.ticker: dict(
                mean_expectancy=portfolio.trades.expectancy().mean(),
                max_expectancy=portfolio.trades.expectancy().max(),
                mean_random_expectancy=random_porfolio.trades.expectancy().mean(),
                mean_permuted_expectancy=permuted_portfolio.trades.expectancy().mean(),
            )
        }
        return output

In [51]:
class Backtest_V2:
    def __init__(
        self,
        ticker: str,
        strategy: Strategy,
        fees: float = 0.001,
        size: float = np.inf,
        freq: str = "1D",
    ):
        self.ticker = ticker
        self.strategy = strategy
        self.fees = fees
        self.size = size
        self.freq = freq
        self.data = self.strategy.data

    def __repr__(self):
        return f"<Bactest {str(self.strategy)}>"

    def _get_portfolio(self, entries, exits, **kwargs):
        return vbt.Portfolio.from_signals(
            close=self.data,
            entries=entries,
            exits=exits,
            fees=self.fees,
            size=self.size,
            freq=self.freq,
            **kwargs
        )

    def run(self, **kwargs):
        entries = self.strategy.get_entries()
        exits = self.strategy.get_exits()
        
        random_entries = entries.apply(random_like)
        permuted_entries = entries.apply(permute_data)
        
        time_based_signal= TimeBasedSignals.run(entries, [50, 100, 200]) # entry vs time_based
        random_vs_time_based_signal = TimeBasedSignals.run(random_entries, [50, 100, 200]) # random_entry vs timebased 
        
        step = 0.01  # in %
        stops = np.arange(step, 1 + step, step)
        entry_vs_trailing_sl = vbt.STCX.run(entries, self.data, 
                                            stop =  list(stops), trailing = True)
        
        random_vs_time_based_entries = random_vs_time_based_signal.new_entries
        random_vs_time_based_exits = random_vs_time_based_signal.exits
        
        time_based_entries = time_based_signal.new_entries #This adapts entries to the exit signals
        time_based_exits = time_based_signal.exits
        
        entry_vs_trailing_sl_entries =  entry_vs_trailing_sl.new_entries
        entry_vs_trailing_sl_exits =  entry_vs_trailing_sl.exits
        
        portfolio = self._get_portfolio(entries,exits, **kwargs)
        random_porfolio = self._get_portfolio(random_entries, exits, **kwargs)
        permuted_portfolio = self._get_portfolio(permuted_entries, exits, **kwargs)
        time_based_pf = self._get_portfolio(time_based_entries, time_based_exits, **kwargs)
        random_vs_time_based_pf = self._get_portfolio(random_vs_time_based_entries,  random_vs_time_based_exits, **kwargs)
        
        time_based_pf  = time_based_pf.trades.expectancy().groupby("custom_num_days").mean()
        random_vs_time_based_pf  = random_vs_time_based_pf.trades.expectancy().groupby("custom_num_days").mean()
        entry_vs_trailing_sl_pf = vbt.Portfolio.from_signals(self.data, entry_vs_trailing_sl_entries,
                                                             entry_vs_trailing_sl_exits)
        
        
        output = {
            self.ticker: dict(
                entry_vs_exit_expectancy=portfolio.trades.expectancy().mean(),
                max_entry_expectancy=portfolio.trades.expectancy().max(),
                random_vs_exit_expectancy =random_porfolio.trades.expectancy().mean(),
                mean_permuted_expectancy=permuted_portfolio.trades.expectancy().mean(),
                entry_vs_50D_expectancy = time_based_pf.values[0],
                entry_vs_100D_expectancy = time_based_pf.values[1],
                entry_vs_200D_expectancy= time_based_pf.values[2],
                random_vs_50D_expectancy = random_vs_time_based_pf.values[0],
                random_vs_100D_expectancy = random_vs_time_based_pf.values[1],
                random_vs_200D_expectancy = random_vs_time_based_pf.values[2],
                entry_vs_trailing_sl_expectancy = entry_vs_trailing_sl_pf.trades.expectancy().mean()
                

            )
        }
        return output

## Strategies

## SMA

In [52]:
class SMAStrategy(Strategy):
    def __init__(self, data):
        self.data = data
    
    def init(self):
        windows=  np.arange(10, 100, 5)
        indicator = vbt.IndicatorFactory.from_pandas_ta("SMA")
        self.fast_sma, self.slow_sma = indicator.run_combs(self.data, windows, short_names=['fast', 'slow'])
        
    def get_entries(self):
        self.init()
        return self.fast_sma.sma_crossed_above(self.slow_sma)
    
    def get_exits(self):
        return self.fast_sma.sma_crossed_below(self.slow_sma)   

#### TODO TEST ON PARTITIONS OF DATA

## MACD

In [53]:
class MACDStrategy(Strategy):
    def __init__(self, data):
        self.data = data
        
    def init(self):
        fast_windows, slow_windows, signal_windows = vbt.utils.params.create_param_combs(
            (product, (combinations, np.arange(2, 51, 1), 2), np.arange(2, 21, 1)))
        
        self.indicator = vbt.MACD.run(
            self.data,
            fast_window=fast_windows,
            slow_window=slow_windows,
            signal_window=signal_windows
        )
    
    def get_entries(self):
        self.init()
        return self.indicator.macd_above(0) & self.indicator.macd_above(self.indicator.signal)
    
    def get_exits(self):
        return self.indicator.macd_below(0) | self.indicator.macd_below(self.indicator.signal)   

In [54]:
## TODO Refactor the code
def run_backtest(ticker:str):
    print(f"backtesting on {ticker}")
    data = pd.read_parquet(f"{DATA_PATH/ticker}.parquet").get("Close")
    strategy = MACDStrategy(data)
    backtest = Backtest(ticker, strategy)
    result = backtest.run()
    return pd.DataFrame.from_dict(result, orient="index")

## BBANDS

In [55]:
class BBANDStrategy(Strategy):
    def __init__(self, data):
        self.data = data
        
    def init(self):
        lengths, stds, mas = vbt.utils.params.create_param_combs((product, np.arange(10, 55, 5),  (product, [2, 3],["sma", "ema"]) ) ) 
        self.indicator = vbt.IndicatorFactory.from_pandas_ta("BBANDS").run(
            self.data,
            length=lengths,
            std=stds,
            mamode = mas
        )
        
    
    def get_entries(self):
        self.init()
        return self.indicator.close_below(self.indicator.bbl)
        
       
    
    def get_exits(self):
        return self.indicator.close_above(self.indicator.bbu)

In [56]:
## ENTRY BUT EXIT AFTER 50, 100, 200 DAYS
#adapted from the documentation

@njit
def wait_choice_nb(from_i, to_i, col, num_days, temp_idx_arr):
    temp_idx_arr[0] = from_i + num_days - 1
    if temp_idx_arr[0] < to_i:
        return temp_idx_arr[:1]
    return temp_idx_arr[:0]  
            
    
# Build signal generator
TimeBasedSignals = SignalFactory(
    mode='chain',
    param_names=['num_days']
).from_choice_func(
    exit_choice_func=wait_choice_nb,
    exit_settings=dict(
        pass_params=['num_days'],
        pass_kwargs=['temp_idx_arr']
    ) 
)

In [129]:
def run_backtest(ticker:str, strategy_object:Strategy,START_DATE=START_DATE,END_DATE=END_DATE):
    data = pd.read_parquet(f"{DATA_PATH/ticker}.parquet").get("Close")
    data = data.loc[START_DATE:END_DATE]
    strategy = strategy_object(data)
    backtest = Backtest_V2(ticker, strategy)
    print(f"backtesting on {ticker}")
    result = backtest.run()
    time.sleep(5)
    return pd.DataFrame.from_dict(result, orient="index")

## Test run

## For entire time period

In [185]:
START_DATE_1  = pd.to_datetime("2010-01-01")
END_DATE_1   = pd.to_datetime("2013-01-01")

START_DATE_2 = pd.to_datetime("2013-01-01")
END_DATE_2 = pd.to_datetime("2016-01-01")

START_DATE_3 = pd.to_datetime("2016-01-01")
END_DATE_3 = pd.to_datetime("2019-01-01")

# PARTITION_1 = titan_price.loc[START_DATE_1 : END_DATE_1]
# PARTITION_2 = titan_price.loc[START_DATE_2 : END_DATE_2]
# PARTITION_3 = titan_price.loc[START_DATE_3 : END_DATE_3]

In [186]:
from functools import partial

In [192]:
sma_strategy = SMAStrategy
sma_backtest = partial(run_backtest, strategy_object = sma_strategy) 

In [193]:
bbands_strategy = BBANDStrategy 
bbands_backtest = partial(run_backtest, strategy_object = bbands_strategy) 

In [194]:
num_batches = 30//3

In [195]:
# for batch in range(num_batches):
#     with ProcessPoolExecutor() as executor:
#         result = executor.map(sma_backtest, tickers[3 * batch: 3 * (batch + 1) ])
#     time.sleep(10)
#     output = pd.concat(list(result))
#     output.to_csv(f"SMA_FULL_{batch}")
import gc

In [196]:

for batch in range(num_batches):
    results = []
    for ticker in tickers[3 * batch : 3 * (batch + 1)]:
        results.append(sma_backtest(ticker))
    results = pd.concat(results)
    print(f"saving for batch {batch}")
    results.to_csv(f"SMA_{batch}.csv")
    gc.collect()

backtesting on NTPC.NS
backtesting on HINDALCO.NS
backtesting on BHARTIARTL.NS
saving for batch 0
backtesting on SHREECEM.NS
backtesting on TCS.NS
backtesting on HDFCLIFE.NS
saving for batch 1
backtesting on CIPLA.NS
backtesting on LT.NS
backtesting on ULTRACEMCO.NS
saving for batch 2
backtesting on WIPRO.NS
backtesting on KOTAKBANK.NS
backtesting on BAJFINANCE.NS
saving for batch 3
backtesting on TATACONSUM.NS
backtesting on BAJAJFINSV.NS
backtesting on MARUTI.NS
saving for batch 4
backtesting on TITAN.NS
backtesting on ICICIBANK.NS
backtesting on ONGC.NS
saving for batch 5
backtesting on ITC.NS
backtesting on APOLLOHOSP.NS
backtesting on BRITANNIA.NS
saving for batch 6
backtesting on BAJAJ-AUTO.NS
backtesting on TECHM.NS
backtesting on COALINDIA.NS
saving for batch 7
backtesting on TATASTEEL.NS
backtesting on HEROMOTOCO.NS
backtesting on INDUSINDBK.NS
saving for batch 8
backtesting on NESTLEIND.NS
backtesting on M&M.NS
backtesting on RELIANCE.NS
saving for batch 9


In [197]:

for batch in range(num_batches):
    results = []
    for ticker in tickers[3 * batch : 3 * (batch + 1)]:
        results.append(bbands_backtest(ticker))
    results = pd.concat(results)
    print(f"saving for batch {batch}")
    results.to_csv(f"BB_{batch}.csv")
    gc.collect()

backtesting on NTPC.NS
backtesting on HINDALCO.NS
backtesting on BHARTIARTL.NS
saving for batch 0
backtesting on SHREECEM.NS
backtesting on TCS.NS
backtesting on HDFCLIFE.NS
saving for batch 1
backtesting on CIPLA.NS
backtesting on LT.NS
backtesting on ULTRACEMCO.NS
saving for batch 2
backtesting on WIPRO.NS
backtesting on KOTAKBANK.NS
backtesting on BAJFINANCE.NS
saving for batch 3
backtesting on TATACONSUM.NS
backtesting on BAJAJFINSV.NS
backtesting on MARUTI.NS
saving for batch 4
backtesting on TITAN.NS
backtesting on ICICIBANK.NS
backtesting on ONGC.NS
saving for batch 5
backtesting on ITC.NS
backtesting on APOLLOHOSP.NS
backtesting on BRITANNIA.NS
saving for batch 6
backtesting on BAJAJ-AUTO.NS
backtesting on TECHM.NS
backtesting on COALINDIA.NS
saving for batch 7
backtesting on TATASTEEL.NS
backtesting on HEROMOTOCO.NS
backtesting on INDUSINDBK.NS
saving for batch 8
backtesting on NESTLEIND.NS
backtesting on M&M.NS
backtesting on RELIANCE.NS
saving for batch 9


In [198]:
import glob
sma_output = []
for file in glob.glob('SMA_*'):
  sma_output.append(pd.read_csv(file))

sma_output = pd.concat(sma_output)
sma_output.to_csv(f"SMA.csv")

In [199]:
import glob
bb_output = []
for file in glob.glob('BB_*'):
  bb_output.append(pd.read_csv(file))

bb_output = pd.concat(bb_output)
bb_output.to_csv(f"BB.csv")

In [200]:
bb_output

,Unnamed: 0,entry_vs_exit_expectancy,max_entry_expectancy,random_vs_exit_expectancy,mean_permuted_expectancy,entry_vs_50D_expectancy,entry_vs_100D_expectancy,entry_vs_200D_expectancy,random_vs_50D_expectancy,random_vs_100D_expectancy,random_vs_200D_expectancy,entry_vs_trailing_sl_expectancy,Unnamed: 0.1,Unnamed: 0.1.1
0,WIPRO.NS,10.106659,44.856027,15.573151,8.117826,-1.582306,4.892797,6.221860,1.340559,2.056255,6.722597,24.413284,NaN,NaN
1,KOTAKBANK.NS,28.328570,127.184926,80.508171,21.308686,11.441258,23.062476,56.007797,14.684554,24.200742,62.105229,76.512220,NaN,NaN
2,BAJFINANCE.NS,54.795473,239.507510,1517.458324,51.104874,19.313604,34.165270,235.679500,243.472329,500.351011,1118.542076,112.248251,NaN,NaN
0,0,0.106213,6.016940,2.525390,2.074315,1.206574,2.058313,2.725362,0.886511,2.020592,3.432561,0.412032,WIPRO.NS,NaN
1,1,2.468478,14.365842,11.483937,2.777601,3.659016,13.388640,18.265544,4.345872,7.253672,18.025961,31.161874,KOTAKBANK.NS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,LT.NS,9.004932,55.915223,11.869219,8.291629,1.455503,2.868450,13.686070,1.923074,2.835669,10.293324,15.743398,NaN,NaN
2,ULTRACEMCO.NS,22.047488,75.492314,60.453890,30.607947,6.287164,12.439102,23.156507,5.938413,12.046205,36.849585,55.432663,NaN,NaN
0,BAJAJ-AUTO.NS,18.244812,49.317437,62.053040,22.775830,7.718957,11.064131,22.545859,6.128915,14.473251,30.340575,46.886681,NaN,NaN
1,TECHM.NS,7.952668,60.819198,36.509173,7.714265,2.298460,6.497970,12.774999,4.242795,8.957227,20.394645,26.541949,NaN,NaN


In [201]:
sma_output

,Unnamed: 0,entry_vs_exit_expectancy,max_entry_expectancy,random_vs_exit_expectancy,mean_permuted_expectancy,entry_vs_50D_expectancy,entry_vs_100D_expectancy,entry_vs_200D_expectancy,random_vs_50D_expectancy,random_vs_100D_expectancy,random_vs_200D_expectancy,entry_vs_trailing_sl_expectancy,Unnamed: 0.1
0,TATASTEEL.NS,3.096205,16.261945,0.896600,2.614588,2.325836,-1.726282,2.997770,-0.397573,-0.758579,0.706589,-3.834548,NaN
1,HEROMOTOCO.NS,0.150272,2.750511,8.843765,2.369328,0.270602,4.918700,8.655915,1.982847,4.231878,8.597985,20.911644,NaN
2,INDUSINDBK.NS,2.554820,18.975309,93.962398,12.949800,4.463445,12.739398,21.960385,23.974314,38.563252,81.634568,39.253171,NaN
0,BAJAJ-AUTO.NS,1.644140,7.984471,36.694183,7.728559,4.017105,9.387406,16.960048,6.128915,14.473251,30.340575,53.188070,NaN
1,TECHM.NS,9.088918,19.419504,23.739383,9.225194,5.576750,9.059503,18.906744,4.242795,8.957227,20.394645,35.988421,NaN
2,COALINDIA.NS,-1.015684,1.246059,-1.382413,0.119915,-0.336169,-0.729572,-1.783055,-0.136951,-0.263069,-1.654497,-2.120699,NaN
0,TITAN.NS,13.333030,34.713323,172.527836,38.233163,7.756364,15.943643,46.545806,35.419764,68.247265,139.554163,60.623526,NaN
1,ICICIBANK.NS,2.364691,7.370202,21.654329,4.872986,3.173315,6.578419,20.026726,5.475332,8.914349,17.731077,33.613647,NaN
2,ONGC.NS,0.378148,4.399618,-1.906684,0.882735,-1.484551,-2.347068,-0.248368,-0.679156,-1.304538,-1.622843,-3.608576,NaN
0,CIPLA.NS,-0.233941,1.278630,5.352899,1.188776,1.789828,-0.700274,3.682019,0.927777,2.985203,5.507765,3.480545,NaN


In [202]:
import scipy.stats as ss 


def ab_test(control,test):
  t_stat, p_val= ss.ttest_ind(test,control)
  print("t-value: ", t_stat , "p-value: ", p_val)

In [203]:
# SMA entry vs random 50 days 
n = 50
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  -2.0243201747567756 p-value:  0.04523435477521745


In [204]:
# SMA entry vs random 100 days 
n = 100
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  -1.5570494186252106 p-value:  0.12218291304399452


In [205]:
# SMA entry vs random 200 days 
n = 200
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  -1.4605664558374039 p-value:  0.1468377133010393


In [206]:
# BBands entry vs random 50 days 
n = 50
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  -2.5816989067452965 p-value:  0.010645467738691369


In [207]:
# BBands entry vs random 100 days 
n = 100
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  -2.1218349808082735 p-value:  0.03525094876706672


In [208]:
# BBands entry vs random 200 days 
n = 200
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  -1.9597632225766513 p-value:  0.05160256785021188


In [209]:
# SMA entry-exit vs random-entry-planed-exit

test = f"entry_vs_exit_expectancy"
control = f"random_vs_exit_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  -1.9168756906470714 p-value:  0.05771359131292993


In [210]:
# BBands entry-exit vs random-entry-planed-exit

test = f"entry_vs_exit_expectancy"
control = f"random_vs_exit_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  -2.6991959082483654 p-value:  0.007628341525513335


In [211]:
# SMA entry_trailing_sl_expectancy vs 

## For time period T1

In [212]:

END_DATE_1   = pd.to_datetime("2013-01-01")

# PARTITION_1 = titan_price.loc[START_DATE_1 : END_DATE_1]
# PARTITION_2 = titan_price.loc[START_DATE_2 : END_DATE_2]
# PARTITION_3 = titan_price.loc[START_DATE_3 : END_DATE_3]

In [213]:
from functools import partial

In [214]:
sma_strategy = SMAStrategy
sma_backtest = partial(run_backtest, strategy_object = sma_strategy, START_DATE=START_DATE, END_DATE=END_DATE_1) 

In [215]:
bbands_strategy = BBANDStrategy 
bbands_backtest = partial(run_backtest, strategy_object = bbands_strategy, START_DATE=START_DATE, END_DATE=END_DATE_1) 

In [216]:
num_batches = 30//3

In [217]:
# for batch in range(num_batches):
#     with ProcessPoolExecutor() as executor:
#         result = executor.map(sma_backtest, tickers[3 * batch: 3 * (batch + 1) ])
#     time.sleep(10)
#     output = pd.concat(list(result))
#     output.to_csv(f"SMA_FULL_{batch}")
import gc

In [218]:

for batch in range(num_batches):
    results = []
    for ticker in tickers[3 * batch : 3 * (batch + 1)]:
        try:
          results.append(sma_backtest(ticker))
        except Exception:
          pass
    results = pd.concat(results)
    print(f"saving for batch {batch}")
    results.to_csv(f"SMA_{batch}.csv")
    gc.collect()
       

backtesting on NTPC.NS
backtesting on HINDALCO.NS
backtesting on BHARTIARTL.NS
saving for batch 0
backtesting on SHREECEM.NS
backtesting on TCS.NS
backtesting on HDFCLIFE.NS
saving for batch 1
backtesting on CIPLA.NS
backtesting on LT.NS
backtesting on ULTRACEMCO.NS
saving for batch 2
backtesting on WIPRO.NS
backtesting on KOTAKBANK.NS
backtesting on BAJFINANCE.NS
saving for batch 3
backtesting on TATACONSUM.NS
backtesting on BAJAJFINSV.NS
backtesting on MARUTI.NS
saving for batch 4
backtesting on TITAN.NS
backtesting on ICICIBANK.NS
backtesting on ONGC.NS
saving for batch 5
backtesting on ITC.NS
backtesting on APOLLOHOSP.NS
backtesting on BRITANNIA.NS
saving for batch 6
backtesting on BAJAJ-AUTO.NS
backtesting on TECHM.NS
backtesting on COALINDIA.NS
saving for batch 7
backtesting on TATASTEEL.NS
backtesting on HEROMOTOCO.NS
backtesting on INDUSINDBK.NS
saving for batch 8
backtesting on NESTLEIND.NS
backtesting on M&M.NS
backtesting on RELIANCE.NS
saving for batch 9


In [219]:

for batch in range(num_batches):
    results = []
    for ticker in tickers[3 * batch : 3 * (batch + 1)]:
        try:
          results.append(bbands_backtest(ticker))
        except Exception:
          pass
    results = pd.concat(results)
    print(f"saving for batch {batch}")
    results.to_csv(f"BB_{batch}.csv")
    gc.collect()
    

backtesting on NTPC.NS
backtesting on HINDALCO.NS
backtesting on BHARTIARTL.NS
saving for batch 0
backtesting on SHREECEM.NS
backtesting on TCS.NS
backtesting on HDFCLIFE.NS
saving for batch 1
backtesting on CIPLA.NS
backtesting on LT.NS
backtesting on ULTRACEMCO.NS
saving for batch 2
backtesting on WIPRO.NS
backtesting on KOTAKBANK.NS
backtesting on BAJFINANCE.NS
saving for batch 3
backtesting on TATACONSUM.NS
backtesting on BAJAJFINSV.NS
backtesting on MARUTI.NS
saving for batch 4
backtesting on TITAN.NS
backtesting on ICICIBANK.NS
backtesting on ONGC.NS
saving for batch 5
backtesting on ITC.NS
backtesting on APOLLOHOSP.NS
backtesting on BRITANNIA.NS
saving for batch 6
backtesting on BAJAJ-AUTO.NS
backtesting on TECHM.NS
backtesting on COALINDIA.NS
saving for batch 7
backtesting on TATASTEEL.NS
backtesting on HEROMOTOCO.NS
backtesting on INDUSINDBK.NS
saving for batch 8
backtesting on NESTLEIND.NS
backtesting on M&M.NS
backtesting on RELIANCE.NS
saving for batch 9


In [220]:
import glob
sma_output = []
for file in glob.glob('SMA_*'):
  sma_output.append(pd.read_csv(file))

sma_output = pd.concat(sma_output)
sma_output.to_csv(f"SMA_T1.csv")

In [221]:
import glob
bb_output = []
for file in glob.glob('BB_*'):
  bb_output.append(pd.read_csv(file))

bb_output = pd.concat(bb_output)
bb_output.to_csv(f"BB_T1.csv")

In [222]:
bb_output

,Unnamed: 0,entry_vs_exit_expectancy,max_entry_expectancy,random_vs_exit_expectancy,mean_permuted_expectancy,entry_vs_50D_expectancy,entry_vs_100D_expectancy,entry_vs_200D_expectancy,random_vs_50D_expectancy,random_vs_100D_expectancy,random_vs_200D_expectancy,entry_vs_trailing_sl_expectancy,Unnamed: 0.1,Unnamed: 0.1.1
0,WIPRO.NS,2.112757,18.203369,5.435761,5.567716,-4.504736,-0.702152,5.373073,0.886511,2.020592,3.432561,2.342736,NaN,NaN
1,KOTAKBANK.NS,22.306122,62.381800,14.055983,12.554837,9.285772,11.205702,25.661460,4.345872,7.253672,18.025961,32.978666,NaN,NaN
2,BAJFINANCE.NS,18.933212,118.242176,62.674855,18.867356,3.709044,14.145247,42.602031,25.893363,46.795208,96.083439,28.575513,NaN,NaN
0,0,0.106213,6.016940,2.525390,2.074315,1.206574,2.058313,2.725362,0.886511,2.020592,3.432561,0.412032,WIPRO.NS,NaN
1,1,2.468478,14.365842,11.483937,2.777601,3.659016,13.388640,18.265544,4.345872,7.253672,18.025961,31.161874,KOTAKBANK.NS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,LT.NS,6.222721,55.915223,-0.115621,3.595905,0.936489,5.573052,14.782561,0.049155,-1.074129,2.369991,7.165371,NaN,NaN
2,ULTRACEMCO.NS,23.165964,127.549727,43.804900,35.711955,4.555643,12.675636,25.713256,6.631826,16.208773,33.638584,32.859118,NaN,NaN
0,BAJAJ-AUTO.NS,13.364548,53.900739,42.971592,29.956122,10.801618,9.798464,21.863920,12.639482,23.549052,46.673554,34.395651,NaN,NaN
1,TECHM.NS,3.116805,29.515348,0.565146,-0.170692,3.709368,3.532740,-1.867049,0.139244,-1.356968,0.621671,10.298046,NaN,NaN


In [223]:
sma_output

,Unnamed: 0,entry_vs_exit_expectancy,max_entry_expectancy,random_vs_exit_expectancy,mean_permuted_expectancy,entry_vs_50D_expectancy,entry_vs_100D_expectancy,entry_vs_200D_expectancy,random_vs_50D_expectancy,random_vs_100D_expectancy,random_vs_200D_expectancy,entry_vs_trailing_sl_expectancy,Unnamed: 0.1
0,TATASTEEL.NS,-2.297569,5.746408,-2.313793,-1.987541,-1.934073,-3.702536,-4.004664,-0.775172,-2.230676,-1.973734,-11.667989,NaN
1,HEROMOTOCO.NS,-1.790902,1.242746,6.621486,1.231377,-1.153647,2.465970,5.584912,0.866196,3.416974,7.680100,11.292106,NaN
2,INDUSINDBK.NS,3.034963,23.364315,69.410530,18.298848,5.916466,17.501950,28.621332,17.653442,33.717738,74.546282,39.092908,NaN
0,BAJAJ-AUTO.NS,7.667008,36.464445,66.612216,25.103125,2.546229,10.668581,26.674412,12.639482,23.549052,46.673554,38.786055,NaN
1,TECHM.NS,-0.953425,3.686375,-0.164723,-0.570373,0.555076,2.479091,3.588354,0.139244,-1.356968,0.621671,10.712437,NaN
2,COALINDIA.NS,1.343952,8.660029,2.033670,3.886680,1.969752,2.740030,0.987622,0.113674,0.909334,0.755555,7.406438,NaN
0,TITAN.NS,12.925171,60.970681,123.423565,42.491347,12.976337,22.587942,46.006512,25.700988,46.280768,85.923831,40.161154,NaN
1,ICICIBANK.NS,1.151067,5.334495,7.616023,3.764295,0.370456,2.934760,8.333232,2.184323,3.084179,7.977177,16.731078,NaN
2,ONGC.NS,-1.778775,1.020031,0.116331,0.647005,-1.230617,-1.210438,0.504933,-0.378275,-0.099235,1.671390,2.244411,NaN
0,CIPLA.NS,0.179986,3.346292,11.478940,2.810596,3.386495,-0.577322,11.037252,3.252671,5.986365,14.745124,19.833669,NaN


In [224]:
import scipy.stats as ss 


def ab_test(control,test):
  t_stat, p_val= ss.ttest_ind(test,control)
  print("t-value: ", t_stat , "p-value: ", p_val)

In [225]:
# SMA entry vs random 50 days 
n = 50
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  -2.3811471151558736 p-value:  0.018916110368957052


In [226]:
# SMA entry vs random 100 days 
n = 100
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  -1.7825371852409975 p-value:  0.07732350083783901


In [227]:
# SMA entry vs random 200 days 
n = 200
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  -2.150832097594171 p-value:  0.03360081879209395


In [228]:
# BBands entry vs random 50 days 
n = 50
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  -2.1211678277572354 p-value:  0.03532400231731022


In [229]:
# BBands entry vs random 100 days 
n = 100
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  -1.6689279194298827 p-value:  0.09692997235022527


In [230]:
# BBands entry vs random 200 days 
n = 200
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  -1.7026595798099189 p-value:  0.09041746992524252


In [231]:
# SMA entry-exit vs random-entry-planed-exit

test = f"entry_vs_exit_expectancy"
control = f"random_vs_exit_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  -4.21177052592653 p-value:  5.085706191128013e-05


In [232]:
# BBands entry-exit vs random-entry-planed-exit

test = f"entry_vs_exit_expectancy"
control = f"random_vs_exit_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  -2.0817558989679568 p-value:  0.03882836134125527


In [233]:
# SMA entry_trailing_sl_expectancy vs 

## For time period T2

In [234]:
START_DATE_2  = pd.to_datetime("2013-01-01")
END_DATE_2   = pd.to_datetime("2016-01-01")

# PARTITION_1 = titan_price.loc[START_DATE_1 : END_DATE_1]
# PARTITION_2 = titan_price.loc[START_DATE_2 : END_DATE_2]
# PARTITION_3 = titan_price.loc[START_DATE_3 : END_DATE_3]

In [235]:
from functools import partial

In [236]:
sma_strategy = SMAStrategy
sma_backtest = partial(run_backtest, strategy_object = sma_strategy, START_DATE=START_DATE_2, END_DATE=END_DATE_2) 

In [237]:
bbands_strategy = BBANDStrategy 
bbands_backtest = partial(run_backtest, strategy_object = bbands_strategy, START_DATE=START_DATE_2, END_DATE=END_DATE_2) 

In [238]:
num_batches = 30//3

In [239]:
# for batch in range(num_batches):
#     with ProcessPoolExecutor() as executor:
#         result = executor.map(sma_backtest, tickers[3 * batch: 3 * (batch + 1) ])
#     time.sleep(10)
#     output = pd.concat(list(result))
#     output.to_csv(f"SMA_FULL_{batch}")
import gc

In [240]:

for batch in range(num_batches):
    results = []
    for ticker in tickers[3 * batch : 3 * (batch + 1)]:
        try:
          results.append(sma_backtest(ticker))
        except Exception:
          pass
    results = pd.concat(results)
    print(f"saving for batch {batch}")
    results.to_csv(f"SMA_{batch}.csv")
    gc.collect()
          

backtesting on NTPC.NS
backtesting on HINDALCO.NS
backtesting on BHARTIARTL.NS
saving for batch 0
backtesting on SHREECEM.NS
backtesting on TCS.NS
backtesting on HDFCLIFE.NS
saving for batch 1
backtesting on CIPLA.NS
backtesting on LT.NS
backtesting on ULTRACEMCO.NS
saving for batch 2
backtesting on WIPRO.NS
backtesting on KOTAKBANK.NS
backtesting on BAJFINANCE.NS
saving for batch 3
backtesting on TATACONSUM.NS
backtesting on BAJAJFINSV.NS
backtesting on MARUTI.NS
saving for batch 4
backtesting on TITAN.NS
backtesting on ICICIBANK.NS
backtesting on ONGC.NS
saving for batch 5
backtesting on ITC.NS
backtesting on APOLLOHOSP.NS
backtesting on BRITANNIA.NS
saving for batch 6
backtesting on BAJAJ-AUTO.NS
backtesting on TECHM.NS
backtesting on COALINDIA.NS
saving for batch 7
backtesting on TATASTEEL.NS
backtesting on HEROMOTOCO.NS
backtesting on INDUSINDBK.NS
saving for batch 8
backtesting on NESTLEIND.NS
backtesting on M&M.NS
backtesting on RELIANCE.NS
saving for batch 9


In [241]:

for batch in range(num_batches):
    results = []
    for ticker in tickers[3 * batch : 3 * (batch + 1)]:
        try:
          results.append(bbands_backtest(ticker))
        except Exception:
          pass
    results = pd.concat(results)
    print(f"saving for batch {batch}")
    results.to_csv(f"BB_{batch}.csv")
    gc.collect()
           

backtesting on NTPC.NS
backtesting on HINDALCO.NS
backtesting on BHARTIARTL.NS
saving for batch 0
backtesting on SHREECEM.NS
backtesting on TCS.NS
backtesting on HDFCLIFE.NS
saving for batch 1
backtesting on CIPLA.NS
backtesting on LT.NS
backtesting on ULTRACEMCO.NS
saving for batch 2
backtesting on WIPRO.NS
backtesting on KOTAKBANK.NS
backtesting on BAJFINANCE.NS
saving for batch 3
backtesting on TATACONSUM.NS
backtesting on BAJAJFINSV.NS
backtesting on MARUTI.NS
saving for batch 4
backtesting on TITAN.NS
backtesting on ICICIBANK.NS
backtesting on ONGC.NS
saving for batch 5
backtesting on ITC.NS
backtesting on APOLLOHOSP.NS
backtesting on BRITANNIA.NS
saving for batch 6
backtesting on BAJAJ-AUTO.NS
backtesting on TECHM.NS
backtesting on COALINDIA.NS
saving for batch 7
backtesting on TATASTEEL.NS
backtesting on HEROMOTOCO.NS
backtesting on INDUSINDBK.NS
saving for batch 8
backtesting on NESTLEIND.NS
backtesting on M&M.NS
backtesting on RELIANCE.NS
saving for batch 9


In [242]:
import glob
sma_output = []
for file in glob.glob('SMA_*'):
  sma_output.append(pd.read_csv(file))

sma_output = pd.concat(sma_output)
sma_output.to_csv(f"SMA_T2.csv")

In [243]:
import glob
bb_output = []
for file in glob.glob('BB_*'):
  bb_output.append(pd.read_csv(file))

bb_output = pd.concat(bb_output)
bb_output.to_csv(f"BB_T2.csv")

In [244]:
bb_output

,Unnamed: 0,entry_vs_exit_expectancy,max_entry_expectancy,random_vs_exit_expectancy,mean_permuted_expectancy,entry_vs_50D_expectancy,entry_vs_100D_expectancy,entry_vs_200D_expectancy,random_vs_50D_expectancy,random_vs_100D_expectancy,random_vs_200D_expectancy,entry_vs_trailing_sl_expectancy,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1
0,WIPRO.NS,12.548173,54.157054,10.886843,6.466543,-2.704171,12.871671,28.637798,2.185485,5.330028,13.761445,20.120530,NaN,NaN,NaN
1,KOTAKBANK.NS,17.333248,121.146299,31.892129,21.787336,7.806482,18.399834,38.443731,6.721763,15.109113,26.419601,31.003872,NaN,NaN,NaN
2,BAJFINANCE.NS,36.298097,128.750750,120.025301,26.495614,11.965440,10.952712,47.025919,13.038208,36.629879,89.192311,34.151684,NaN,NaN,NaN
0,0,2.112757,18.203369,5.435761,5.567716,-4.504736,-0.702152,5.373073,0.886511,2.020592,3.432561,2.342736,WIPRO.NS,NaN,NaN
1,1,22.306122,62.381800,14.055983,12.554837,9.285772,11.205702,25.661460,4.345872,7.253672,18.025961,32.978666,KOTAKBANK.NS,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,LT.NS,-4.916647,21.043493,9.708200,10.686696,-3.241722,-4.597701,-0.530767,1.047172,2.284616,3.327028,-2.427479,NaN,NaN,NaN
2,ULTRACEMCO.NS,9.947089,77.971650,11.846553,4.494180,6.690740,3.488392,10.073668,2.860522,3.936872,8.920740,23.903512,NaN,NaN,NaN
0,BAJAJ-AUTO.NS,9.592131,34.474076,7.381535,10.839348,3.977070,10.114324,17.086024,0.870922,3.498481,6.165690,22.989252,NaN,NaN,NaN
1,TECHM.NS,-3.032091,8.764562,31.746028,22.481806,-0.129805,8.237295,16.962453,8.197708,15.297771,30.268683,10.333393,NaN,NaN,NaN


In [245]:
sma_output

,Unnamed: 0,entry_vs_exit_expectancy,max_entry_expectancy,random_vs_exit_expectancy,mean_permuted_expectancy,entry_vs_50D_expectancy,entry_vs_100D_expectancy,entry_vs_200D_expectancy,random_vs_50D_expectancy,random_vs_100D_expectancy,random_vs_200D_expectancy,entry_vs_trailing_sl_expectancy,Unnamed: 0.1,Unnamed: 0.1.1
0,TATASTEEL.NS,6.956242,18.648585,-15.099694,-0.445904,9.320486,2.802440,16.858324,-3.121703,-4.202780,-8.408317,5.595004,NaN,NaN
1,HEROMOTOCO.NS,8.360055,17.388777,18.754404,6.800356,7.183115,17.137281,25.395640,1.913318,5.929802,14.044366,27.543554,NaN,NaN
2,INDUSINDBK.NS,6.579583,19.839138,36.690587,10.504614,4.315732,6.694364,19.877154,5.704746,14.847584,31.475392,19.030130,NaN,NaN
0,BAJAJ-AUTO.NS,2.368794,11.117859,7.356831,3.720087,5.164145,7.970299,9.133063,0.870922,3.498481,6.165690,27.539339,NaN,NaN
1,TECHM.NS,24.375564,50.376045,38.275359,15.463742,11.914198,27.747253,39.243324,8.197708,15.297771,30.268683,28.865819,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,WIPRO.NS,2.935645,10.419349,11.604484,6.284983,5.781580,4.650456,10.326389,2.185485,5.330028,13.761445,23.030301,NaN,NaN
1,KOTAKBANK.NS,8.066168,22.796453,33.132522,13.515823,4.428819,8.868641,22.760295,6.721763,15.109113,26.419601,22.053345,NaN,NaN
2,BAJFINANCE.NS,62.805410,176.817685,168.020648,65.377372,7.239885,13.996479,41.235350,13.038208,36.629879,89.192311,22.877467,NaN,NaN
0,SHREECEM.NS,13.237881,35.658277,41.147610,21.977208,2.399492,9.372691,16.623854,7.159805,16.996425,36.411605,17.132219,NaN,NaN


In [246]:
import scipy.stats as ss 


def ab_test(control,test):
  t_stat, p_val= ss.ttest_ind(test,control)
  print("t-value: ", t_stat , "p-value: ", p_val)

In [247]:
# SMA entry vs random 50 days 
n = 50
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  -1.8988519574205436 p-value:  0.059258165475359746


In [248]:
# SMA entry vs random 100 days 
n = 100
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  -1.7150230351880107 p-value:  0.08814197010092238


In [249]:
# SMA entry vs random 200 days 
n = 200
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  -2.0381042646568344 p-value:  0.04307101126972418


In [250]:
# BBands entry vs random 50 days 
n = 50
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  -1.9493347203631495 p-value:  0.05246059731162139


In [251]:
# BBands entry vs random 100 days 
n = 100
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  -1.614172995717998 p-value:  0.10784914594722365


In [252]:
# BBands entry vs random 200 days 
n = 200
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  -1.6663248800570105 p-value:  0.09699823709600827


In [253]:
# SMA entry-exit vs random-entry-planed-exit

test = f"entry_vs_exit_expectancy"
control = f"random_vs_exit_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  -4.394403579325428 p-value:  1.9383111330843938e-05


In [254]:
# BBands entry-exit vs random-entry-planed-exit

test = f"entry_vs_exit_expectancy"
control = f"random_vs_exit_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  -2.2194695616501887 p-value:  0.02742267129141231


In [255]:
# SMA entry_trailing_sl_expectancy vs 

## For time period T3

In [262]:
START_DATE_3  = pd.to_datetime("2016-01-01")
END_DATE_3   = pd.to_datetime("2019-01-01")

# PARTITION_1 = titan_price.loc[START_DATE_1 : END_DATE_1]
# PARTITION_2 = titan_price.loc[START_DATE_2 : END_DATE_2]
# PARTITION_3 = titan_price.loc[START_DATE_3 : END_DATE_3]

In [263]:
from functools import partial

In [264]:
sma_strategy = SMAStrategy
sma_backtest = partial(run_backtest, strategy_object = sma_strategy, START_DATE=START_DATE_3, END_DATE=END_DATE_3) 

In [265]:
bbands_strategy = BBANDStrategy 
bbands_backtest = partial(run_backtest, strategy_object = bbands_strategy, START_DATE=START_DATE_3, END_DATE=END_DATE_3) 

In [266]:
num_batches = 30//3

In [267]:
# for batch in range(num_batches):
#     with ProcessPoolExecutor() as executor:
#         result = executor.map(sma_backtest, tickers[3 * batch: 3 * (batch + 1) ])
#     time.sleep(10)
#     output = pd.concat(list(result))
#     output.to_csv(f"SMA_FULL_{batch}")
import gc

In [268]:

for batch in range(num_batches):
    results = []
    for ticker in tickers[3 * batch : 3 * (batch + 1)]:
        try:
          results.append(sma_backtest(ticker))
        except Exception:
          pass
    results = pd.concat(results)
    print(f"saving for batch {batch}")
    results.to_csv(f"SMA_{batch}.csv")
    gc.collect()
    

backtesting on NTPC.NS
backtesting on HINDALCO.NS
backtesting on BHARTIARTL.NS
saving for batch 0
backtesting on SHREECEM.NS
backtesting on TCS.NS
backtesting on HDFCLIFE.NS
saving for batch 1
backtesting on CIPLA.NS
backtesting on LT.NS
backtesting on ULTRACEMCO.NS
saving for batch 2
backtesting on WIPRO.NS
backtesting on KOTAKBANK.NS
backtesting on BAJFINANCE.NS
saving for batch 3
backtesting on TATACONSUM.NS
backtesting on BAJAJFINSV.NS
backtesting on MARUTI.NS
saving for batch 4
backtesting on TITAN.NS
backtesting on ICICIBANK.NS
backtesting on ONGC.NS
saving for batch 5
backtesting on ITC.NS
backtesting on APOLLOHOSP.NS
backtesting on BRITANNIA.NS
saving for batch 6
backtesting on BAJAJ-AUTO.NS
backtesting on TECHM.NS
backtesting on COALINDIA.NS
saving for batch 7
backtesting on TATASTEEL.NS
backtesting on HEROMOTOCO.NS
backtesting on INDUSINDBK.NS
saving for batch 8
backtesting on NESTLEIND.NS
backtesting on M&M.NS
backtesting on RELIANCE.NS
saving for batch 9


In [269]:

for batch in range(num_batches):
    results = []
    for ticker in tickers[3 * batch : 3 * (batch + 1)]:
        try:
          results.append(bbands_backtest(ticker))
        except Exception:
          pass
    results = pd.concat(results)
    print(f"saving for batch {batch}")
    results.to_csv(f"BB_{batch}.csv")
    gc.collect()
      

backtesting on NTPC.NS
backtesting on HINDALCO.NS
backtesting on BHARTIARTL.NS
saving for batch 0
backtesting on SHREECEM.NS
backtesting on TCS.NS
backtesting on HDFCLIFE.NS
saving for batch 1
backtesting on CIPLA.NS
backtesting on LT.NS
backtesting on ULTRACEMCO.NS
saving for batch 2
backtesting on WIPRO.NS
backtesting on KOTAKBANK.NS
backtesting on BAJFINANCE.NS
saving for batch 3
backtesting on TATACONSUM.NS
backtesting on BAJAJFINSV.NS
backtesting on MARUTI.NS
saving for batch 4
backtesting on TITAN.NS
backtesting on ICICIBANK.NS
backtesting on ONGC.NS
saving for batch 5
backtesting on ITC.NS
backtesting on APOLLOHOSP.NS
backtesting on BRITANNIA.NS
saving for batch 6
backtesting on BAJAJ-AUTO.NS
backtesting on TECHM.NS
backtesting on COALINDIA.NS
saving for batch 7
backtesting on TATASTEEL.NS
backtesting on HEROMOTOCO.NS
backtesting on INDUSINDBK.NS
saving for batch 8
backtesting on NESTLEIND.NS
backtesting on M&M.NS
backtesting on RELIANCE.NS
saving for batch 9


In [270]:
import glob
sma_output = []
for file in glob.glob('SMA_*'):
  sma_output.append(pd.read_csv(file))

sma_output = pd.concat(sma_output)
sma_output.to_csv(f"SMA_T3.csv")

In [271]:
import glob
bb_output = []
for file in glob.glob('BB_*'):
  bb_output.append(pd.read_csv(file))

bb_output = pd.concat(bb_output)
bb_output.to_csv(f"BB_T3.csv")

In [272]:
bb_output

,Unnamed: 0,entry_vs_exit_expectancy,max_entry_expectancy,random_vs_exit_expectancy,mean_permuted_expectancy,entry_vs_50D_expectancy,entry_vs_100D_expectancy,entry_vs_200D_expectancy,random_vs_50D_expectancy,random_vs_100D_expectancy,random_vs_200D_expectancy,entry_vs_trailing_sl_expectancy,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1
0,WIPRO.NS,9.486702,42.443693,6.646625,7.836186,-1.807092,2.249141,10.554416,1.798404,3.716760,6.341274,21.662482,NaN,NaN,NaN,NaN
1,KOTAKBANK.NS,12.480859,79.790255,18.881293,2.028980,6.891157,15.174639,20.838557,3.968371,8.064855,19.044153,28.565285,NaN,NaN,NaN,NaN
2,BAJFINANCE.NS,13.733953,123.366333,64.875423,9.455923,4.594526,18.527367,72.898332,23.421341,40.001427,95.262819,30.827351,NaN,NaN,NaN,NaN
0,0,2.112757,18.203369,5.435761,5.567716,-4.504736,-0.702152,5.373073,0.886511,2.020592,3.432561,2.342736,WIPRO.NS,NaN,NaN,NaN
1,1,22.306122,62.381800,14.055983,12.554837,9.285772,11.205702,25.661460,4.345872,7.253672,18.025961,32.978666,KOTAKBANK.NS,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,LT.NS,10.476839,39.629347,14.676689,9.025505,8.785782,13.337848,23.463560,5.645706,9.373598,19.547526,34.518428,NaN,NaN,NaN,NaN
2,ULTRACEMCO.NS,9.483951,32.640624,16.261357,8.533467,1.021743,8.911025,7.849197,1.893395,5.372646,10.727220,8.719595,NaN,NaN,NaN,NaN
0,BAJAJ-AUTO.NS,9.781977,24.006523,6.905191,5.629463,4.339216,6.903252,8.950837,0.196792,1.611715,3.564334,13.427885,NaN,NaN,NaN,NaN
1,TECHM.NS,17.294396,98.196522,11.448930,17.728354,2.001582,7.004009,6.917758,2.980916,4.922934,10.986955,25.702821,NaN,NaN,NaN,NaN


In [273]:
sma_output

,Unnamed: 0,entry_vs_exit_expectancy,max_entry_expectancy,random_vs_exit_expectancy,mean_permuted_expectancy,entry_vs_50D_expectancy,entry_vs_100D_expectancy,entry_vs_200D_expectancy,random_vs_50D_expectancy,random_vs_100D_expectancy,random_vs_200D_expectancy,entry_vs_trailing_sl_expectancy,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1
0,TATASTEEL.NS,7.319499,25.307170,38.398159,8.071981,5.661142,8.735415,12.784000,7.680151,16.008709,30.287266,14.729127,NaN,NaN,NaN
1,HEROMOTOCO.NS,1.359679,7.105270,6.379074,4.399980,2.009967,3.348691,0.113485,2.032641,2.822621,6.537485,4.927700,NaN,NaN,NaN
2,INDUSINDBK.NS,11.544437,24.086331,24.900228,7.991995,8.434702,16.424926,9.957655,3.848753,6.536779,15.761356,25.954899,NaN,NaN,NaN
0,BAJAJ-AUTO.NS,-0.721690,4.041290,3.446639,1.919533,1.360819,1.663090,3.738013,0.196792,1.611715,3.564334,7.666974,NaN,NaN,NaN
1,TECHM.NS,7.995258,16.836241,15.107631,13.384572,2.938592,3.137669,13.233095,2.980916,4.922934,10.986955,13.112010,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0,13.237881,35.658277,41.147610,21.977208,2.399492,9.372691,16.623854,7.159805,16.996425,36.411605,17.132219,SHREECEM.NS,NaN,NaN
86,1,3.649775,12.523051,20.175928,5.007960,5.204380,9.887888,18.982525,6.385921,12.782101,25.727164,26.207147,TCS.NS,NaN,NaN
0,SHREECEM.NS,-1.928247,7.854027,12.093482,5.385288,0.221926,1.014858,2.090344,2.811884,5.650020,13.587040,9.620414,NaN,NaN,NaN
1,TCS.NS,9.507913,60.243036,15.732794,8.637088,1.078197,6.155508,15.048865,4.045339,8.122061,17.854395,29.296890,NaN,NaN,NaN


In [274]:
import scipy.stats as ss 


def ab_test(control,test):
  t_stat, p_val= ss.ttest_ind(test,control)
  print("t-value: ", t_stat , "p-value: ", p_val)

In [275]:
# SMA entry vs random 50 days 
n = 50
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  -2.8864715285105578 p-value:  0.004139135744901136


In [276]:
# SMA entry vs random 100 days 
n = 100
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  -2.2686160980697188 p-value:  0.023903689019764172


In [277]:
# SMA entry vs random 200 days 
n = 200
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  -3.0963520490134044 p-value:  0.002118227040363561


In [278]:
# BBands entry vs random 50 days 
n = 50
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  -2.866202426131002 p-value:  0.00434149433765045


In [279]:
# BBands entry vs random 100 days 
n = 100
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  -2.2469078185720828 p-value:  0.025111209377200152


In [280]:
# BBands entry vs random 200 days 
n = 200
test = f"entry_vs_{n}D_expectancy"
control = f"random_vs_{n}D_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  -2.3693481325740358 p-value:  0.01822433773904623


In [281]:
# SMA entry-exit vs random-entry-planed-exit

test = f"entry_vs_exit_expectancy"
control = f"random_vs_exit_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  -6.577138201728487 p-value:  1.760386163696116e-10


In [282]:
# BBands entry-exit vs random-entry-planed-exit

test = f"entry_vs_exit_expectancy"
control = f"random_vs_exit_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  -3.129941653073346 p-value:  0.0018577087216776467


In [283]:
# SMA entry_trailing_sl_expectancy vs 

In [284]:
# SMA entry-exit vs random-entry-planed-exit

test = f"entry_vs_exit_expectancy"
control = f"random_vs_exit_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  -6.577138201728487 p-value:  1.760386163696116e-10


,Unnamed: 0,entry_vs_exit_expectancy,max_entry_expectancy,random_vs_exit_expectancy,mean_permuted_expectancy,entry_vs_50D_expectancy,entry_vs_100D_expectancy,entry_vs_200D_expectancy,random_vs_50D_expectancy,random_vs_100D_expectancy,random_vs_200D_expectancy,entry_vs_trailing_sl_expectancy,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1
0,TATASTEEL.NS,7.319499,25.307170,38.398159,8.071981,5.661142,8.735415,12.784000,7.680151,16.008709,30.287266,14.729127,NaN,NaN,NaN
1,HEROMOTOCO.NS,1.359679,7.105270,6.379074,4.399980,2.009967,3.348691,0.113485,2.032641,2.822621,6.537485,4.927700,NaN,NaN,NaN
2,INDUSINDBK.NS,11.544437,24.086331,24.900228,7.991995,8.434702,16.424926,9.957655,3.848753,6.536779,15.761356,25.954899,NaN,NaN,NaN
0,BAJAJ-AUTO.NS,-0.721690,4.041290,3.446639,1.919533,1.360819,1.663090,3.738013,0.196792,1.611715,3.564334,7.666974,NaN,NaN,NaN
1,TECHM.NS,7.995258,16.836241,15.107631,13.384572,2.938592,3.137669,13.233095,2.980916,4.922934,10.986955,13.112010,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0,13.237881,35.658277,41.147610,21.977208,2.399492,9.372691,16.623854,7.159805,16.996425,36.411605,17.132219,SHREECEM.NS,NaN,NaN
86,1,3.649775,12.523051,20.175928,5.007960,5.204380,9.887888,18.982525,6.385921,12.782101,25.727164,26.207147,TCS.NS,NaN,NaN
0,SHREECEM.NS,-1.928247,7.854027,12.093482,5.385288,0.221926,1.014858,2.090344,2.811884,5.650020,13.587040,9.620414,NaN,NaN,NaN
1,TCS.NS,9.507913,60.243036,15.732794,8.637088,1.078197,6.155508,15.048865,4.045339,8.122061,17.854395,29.296890,NaN,NaN,NaN


## Entry-Exit signal, Entry-trailing-stoploss-exit

In [286]:

sma_output = pd.read_csv(f"SMA.csv")

bb_output = pd.read_csv(f"BB.csv")

In [289]:
test = f"entry_vs_exit_expectancy"
control = f"entry_vs_50D_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  1.1716268740490856 p-value:  0.2437477622280411


In [290]:
test = f"entry_vs_exit_expectancy"
control = f"entry_vs_50D_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  2.312284442271836 p-value:  0.021919664640777096


In [291]:
test = f"entry_vs_trailing_sl_expectancy"
control = f"entry_vs_50D_expectancy"
ab_test(sma_output[control],sma_output[test])

t-value:  5.729150000411985 p-value:  8.100780825620205e-08


In [292]:
test = f"entry_vs_trailing_sl_expectancy"
control = f"entry_vs_50D_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  7.958542570148352 p-value:  2.054604127393371e-13


In [302]:
test = f"entry_vs_trailing_sl_expectancy"
control = f"entry_vs_exit_expectancy"
ab_test(bb_output[control],bb_output[test])

t-value:  4.789881652099783 p-value:  3.5282219403120583e-06


In [303]:
test = f"entry_vs_trailing_sl_expectancy"
control = f"entry_vs_exit_expectancy"
ab_test(bb_output[test],bb_output[control])

t-value:  -4.789881652099783 p-value:  3.5282219403120583e-06


In [300]:
from scipy.special import rel_entr

In [301]:
test_sma1 = sma_output["entry_vs_trailing_sl_expectancy"]
test_sma2 = sma_output["entry_vs_exit_expectancy"]
test_bb1 = bb_output["entry_vs_trailing_sl_expectancy"]
test_bb2 = bb_output["entry_vs_exit_expectancy"]
control_sma = sma_output["entry_vs_50D_expectancy"]
control_bb = bb_output["entry_vs_50D_expectancy"]
print("SMA")
print("entry-sl vs entry_vs_50D_expectancy", sum(rel_entr(test_sma1, control_sma)))
print("entry-exit vs entry_vs_50D_expectancy", sum(rel_entr(test_sma2, control_sma)))
print("BB")
print("entry-sl vs entry_vs_50D_expectancy", sum(rel_entr(test_bb1, control_bb)))
print("entry-exit vs entry_vs_50D_expectancy", sum(rel_entr(test_bb2, control_bb)))

SMA
entry-sl vs entry_vs_50D_expectancy inf
entry-exit vs entry_vs_50D_expectancy inf
BB
entry-sl vs entry_vs_50D_expectancy inf
entry-exit vs entry_vs_50D_expectancy inf


In [305]:
test_sma1.mean()

33.441951663594985

In [306]:
test_sma2.mean()

9.88352568317621

In [307]:
control_sma.mean()

4.419649420810386

In [309]:
test_bb1.mean()

36.484288105532166

In [310]:
test_bb2.mean()

12.275935516762573

In [308]:
control_bb.mean()

4.83248469910414

In [311]:
control_sma_100 = sma_output["entry_vs_100D_expectancy"]
control_sma_200 = sma_output["entry_vs_200D_expectancy"]
control_bb_100 = bb_output["entry_vs_100D_expectancy"]
control_bb_200 = bb_output["entry_vs_200D_expectancy"]

In [312]:
control_sma_100.mean()

11.538449771276325

In [313]:
control_sma_200.mean()

26.161605871323914

In [314]:
control_bb_100.mean()

11.977102309043696

In [315]:
control_bb_200.mean()

28.235871194350086